## Read .ghg metadata

In [4]:
import preProcessing
import importlib
import time
importlib.reload(preProcessing)
T1 = time.time()

# pr = preProcessing.read_ALL('BB',2023,3,processes=4,Test=48,reset=1)
pr = preProcessing.read_ALL('BB',2023,3,processes=4,reset=1)
T2 = time.time()
print('\n')
print('Runtime: ',(T2-T1)/60,' minutes')


                                                     filename Flag
TIMESTAMP                                                         
2023-03-17 14:12:55  2023-03-17T141255_LI-7200_incomplete.ghg    -
2023-03-17 14:32:15  2023-03-17T143215_LI-7200_incomplete.ghg    -
2023-03-17 15:11:51  2023-03-17T151151_LI-7200_incomplete.ghg    -


Runtime:  0.00859395662943522  minutes


C:/highfreq/BB/metadata/


['File_Inventory.csv']

In [11]:
import setupEP as eP
import importlib
import time
importlib.reload(eP)

########## Note - incomplete ghg files crash program

T1 = time.time()
mR = eP.makeRun('ep_Templates/DefaultSettings.eddypro','BB')
mR.runDates(['2023-03-17 13:00','2023-03-17 16:00'],'test',threads=2,priority = 'high')
T2 = time.time()
print('\n')
print('Runtime: ',(T2-T1)/60)


2
3
Running EddyPro[████████████████████████████████████████████████████████████] 4/4

Runtime:  0.8971825242042542


# Speedtests

All times in minutes

### 4 threads (8 timesteps each)

Normal priority: 2.05
High priority: 1.91

### 4 threads (48 timesteps each x 12 days)

High priority: 29

### Eddypro GUI (1 run [12 days, 576 timesteps])

High priority: 90